In [768]:
# Libraries Used
import numpy as np
import pandas as pd
import copy

In [761]:
### Referenced GeeksforGeeks to read in a txt file https://www.geeksforgeeks.org/how-to-read-space-delimited-files-in-pandas/#
small_data_19 = pd.read_csv('/Users/kevintu/Documents/Python/CS205/CS_205/CS205_small_Data__19.txt', sep='  ', header=None)
### Referenced pandas library documentation for renaming a column https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html
small_data_19 = small_data_19.rename(columns={0 : "label"})
# print(small_data_19)
# print(small_data_19.head(5))

### Referenced GeeksforGeeks to read in a txt file https://www.geeksforgeeks.org/how-to-read-space-delimited-files-in-pandas/#
large_data_6 = pd.read_csv('/Users/kevintu/Documents/Python/CS205/CS_205/CS205_large_Data__6.txt', sep='  ', header=None)
### Referenced pandas library documentation for renaming a column https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html
large_data_6 = large_data_6.rename(columns={0 : "label"})
# print(large_data_6)
# print(large_data_6.head(5))

/var/folders/6j/jnsrc7z51m96yrq49msyk2rh0000gn/T/ipykernel_13204/3497228385.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  small_data_19 = pd.read_csv('/Users/kevintu/Documents/Python/CS205/CS_205/CS205_small_Data__19.txt', sep='  ', header=None)
/var/folders/6j/jnsrc7z51m96yrq49msyk2rh0000gn/T/ipykernel_13204/3497228385.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  large_data_6 = pd.read_csv('/Users/kevintu/Documents/Python/CS205/CS_205/CS205_large_Data__6.txt', sep='  ', header=None)


In [612]:
class KNN_Classifier:
    def __init__(self, train_set=None, test_set=None, k=None, nearest_neighbors=None):
        self.train_set = train_set
        self.test_set = test_set
        self.k = k
        self.nearest_neightbors = nearest_neighbors

    def distance(self, nums):
        sum = 0
        for num in nums:
            sum += num**2
        return np.sqrt(sum)
        
    def pred_class(self):
        pred_classes = []
        for row in range(self.k):
            # print("KNN: ", self.train_set[self.nearest_neighbors[row]])
            # print("KNN: ", self.train_set[self.nearest_neighbors[row]][0])
            pred_classes.append(self.train_set[self.nearest_neighbors[row]][0])
        ### Referenced NumPy unique to return the predicted class https://numpy.org/doc/stable/reference/generated/numpy.unique.html
        classes, counts = np.unique(pred_classes, return_counts=True)
        if len(counts) == 1:
            return classes
        if counts[0] > counts[1]:
            return classes[0]
        return classes[1]
    
    def train(self):
        ### Test every testing entry to training entry
        nearest_neighbors = []
        # print(self.test_set)
        for train_row in self.train_set:
            # print(train_row)
            # for test_row in self.test_set:
            #     print(test_row)
            nearest_neighbors.append(self.distance(train_row[1:] - self.test_set[1:]))
        # print(nearest_neighbors)
        ### Referenced NumPy argsort to sort by index of distances https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
        self.nearest_neighbors = np.argsort(nearest_neighbors)
        # print(self.nearest_neighbors)


In [613]:
### Start by splitting my data into train and test set
data = {'label': [2.0000000e+000, 1.0000000e+000, 2.0000000e+000, 1.0000000e+000, 2],
        'feature_1': [1.2340000e+010, 6.0668000e+000, 2.3400000e+010, 4.5645400e+010, 1],
        'feature_2': [ 2.3440000e+000, 5.0770000e+000, 3.6460000e+000, 3.0045000e+000, 2]}
data = {'label': [2, 1, 2, 1, 2],
        'feature_1': [1, 10, 2, 11, 1],
        'feature_2': [ 2, 11, 3, 12, 2]}
df = pd.DataFrame(data)
# print(df)
train_data = df.sample(frac = 0.8, random_state = 4)
train_data.reset_index(drop=True, inplace=True)
print(train_data)
test_data = df.drop(train_data.index)
test_data.reset_index(drop=True, inplace=True)
print(test_data)

knn = KNN_Classifier(train_data.to_numpy(), test_data.to_numpy(), 3)
knn.train()
print(knn.pred_class())

   label  feature_1  feature_2
0      2          1          2
1      1         11         12
2      1         10         11
3      2          1          2
   label  feature_1  feature_2
0      2          1          2


ValueError: operands could not be broadcast together with shapes (2,) (0,3) 

In [846]:
class Feature_Selection:
    def __init__(self, dataset=None, k=None, feature_set=None):
        self.dataset=dataset
        self.k=k
        self.feature_set = []
        self.feature_subset = pd.DataFrame(dataset['label'])
    
    def print_feature_set(self):
        print(self.feature_set)
        
    def populate_feature_subset(self):
        self.feature_subset = copy.deepcopy(self.dataset)
        
    def greedy_forward(self):
        # self.feature_subset = pd.concat([self.feature_subset, pd.DataFrame(self.dataset[6])], axis=1)
        # self.feature_subset = pd.concat([self.feature_subset, pd.DataFrame(self.dataset[11])], axis=1)
        # self.feature_subset = pd.concat([self.feature_subset, pd.DataFrame(self.dataset[9])], axis=1)
        if self.dataset.shape[1] == 1:
            return
        precision = []
        feature_name = []
        ### Retreiving the Columns for Feature Selection
        for column in self.dataset.columns:
            if column == 'label':
                continue
            # print(column)
            feature = pd.concat([self.feature_subset, pd.DataFrame(self.dataset[column])], axis=1).to_numpy()
            # print(feature)
            correct = 0
            ### K-Fold CV
            for cfv in range(int(len(feature)/self.k)):
                # print(cfv)
                # print(len(feature)/self.k)
                train_data = np.delete(feature, cfv, axis=0)
                # print(train_data)
                test_data = feature[cfv]
                # print(test_data)

                knn = KNN_Classifier(train_data, test_data, 1)
                knn.train()
                # print(test_data[0])
                # print(knn.pred_class()[0])
                if knn.pred_class()[0] == test_data[0]:
                    # print('HELLO')
                    correct += 1
            precision.append(correct/int(len(feature)/self.k))
            feature_name.append(column)
        # print(precision)
        # print(feature_name)
        precision_index = np.argsort(precision)
        # print(precision_index)
        # print("Best feature performed with: ", precision[precision_index[-1]])
        # print("Best feature performed is feature: ", feature_name[precision_index[-1]])
        # print("Selected feature is: ",self.dataset.columns[precision_index[-1]]+1)
        # print("Selected feature is: ",feature_name[precision_index[-1]])
        self.feature_set.append(feature_name[precision_index[-1]])
        print(precision[precision_index[-1]])
        print(self.feature_set)
        ### Referenced Pandas library to understand how to drop column https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
        self.feature_subset = pd.concat([self.feature_subset, pd.DataFrame(self.dataset[feature_name[precision_index[-1]]])], axis=1)
        # print(self.feature_subset)
        self.dataset = self.dataset.drop(columns=[feature_name[precision_index[-1]]])
        # print(self.dataset)
        self.greedy_forward()
        self.print_feature_set
        pass
    
    def greedy_backward(self):
        if self.feature_subset.shape[1] == 1:
            return
        precision = []
        feature_name = []
        ### Retreiving the Columns for Feature Selection
        for column in self.dataset.columns:
            if column == 'label':
                continue
            # print(column)
            feature = self.feature_subset.drop(columns=[column]).to_numpy()
            # print(feature)
            correct = 0
            ### K-Fold CV
            for cfv in range(int(len(feature)/self.k)):
                # print(cfv)
                # print(len(feature)/self.k)
                train_data = np.delete(feature, cfv, axis=0)
                # print(train_data)
                test_data = feature[cfv]
                # print(test_data)

                knn = KNN_Classifier(train_data, test_data, 1)
                knn.train()
                # print(test_data[0])
                # print(knn.pred_class()[0])
                if knn.pred_class()[0] == test_data[0]:
                    # print('HELLO')
                    correct += 1
            precision.append(correct/int(len(feature)/self.k))
            feature_name.append(column)
        # print(precision)
        # print(feature_name)
        precision_index = np.argsort(precision)
        # print(precision_index)
        # print("Best feature performed with: ", precision[precision_index[-1]])
        # print("Best feature performed is feature: ", feature_name[precision_index[-1]])
        # print("Selected feature is: ",self.dataset.columns[precision_index[-1]]+1)
        # print("Selected feature is: ",feature_name[precision_index[-1]])
        print(precision[precision_index[-1]])
        self.feature_set.append(feature_name[precision_index[-1]])
        print(self.feature_set)
        ### Referenced Pandas library to understand how to drop column https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
        self.feature_subset = self.feature_subset.drop(columns=[feature_name[precision_index[-1]]])
        # print(self.feature_subset)
        self.dataset = self.dataset.drop(columns=[feature_name[precision_index[-1]]])
        # print(self.dataset)
        self.greedy_backward()
        self.print_feature_set
        pass

In [849]:
### Start by splitting my data into train and test set
data = {'label': [2, 1, 2, 1, 2],
        'feature_1': [1, 10, 2, 11, 1],
        'feature_2': [ 2, 11, 3, 12, 2],
        'feature_3': [ 15, 15, 15, 15, 15],}
df = pd.DataFrame(data)
# print(df)
# df = small_data_19.head(5)
df = small_data_19
# df = large_data_6
train_data = df.sample(frac = 0.8, random_state = 4)
train_data.reset_index(drop=True, inplace=True)
# print(train_data)
test_data = df.drop(train_data.index)
test_data.reset_index(drop=True, inplace=True)
# print(test_data)

# feat_select = Feature_Selection(train_data, 1)
feat_select = Feature_Selection(df, 1)
feat_select.greedy_forward()
feat_select.populate_feature_subset()
feat_select.greedy_backward()


0.758
[11]
0.784
[11, 5]
0.784
[11, 5, 12]
0.798
[11, 5, 12, 2]
0.826
[11, 5, 12, 2, 8]
0.866
[11, 5, 12, 2, 8, 7]
0.866
[11, 5, 12, 2, 8, 7, 10]
0.892
[11, 5, 12, 2, 8, 7, 10, 3]
0.928
[11, 5, 12, 2, 8, 7, 10, 3, 1]
0.95
[11, 5, 12, 2, 8, 7, 10, 3, 1, 4]
0.84
[11, 5, 12, 2, 8, 7, 10, 3, 1, 4, 9]
0.214
[11, 5, 12, 2, 8, 7, 10, 3, 1, 4, 9, 6]
